# Защита персональных данных клиентов

Необходимо защитить данные клиентов страховой компании. Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Докажем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Наилучшую модель подбирать не будем.

**Содержание:**
1.  Загрузка данных
2.  Умножение матриц \
2-1.  Проведем расчеты собственным алгоритмом и импортированным \
2-2.  Расчеты формулами \
2-3.  Расчеты формулами (v2)
3.  Алгоритм преобразования
4.  Проверка алгоритма

## 1 Загрузка данных

In [1]:
import numpy as np # импорт библиотеки numpy
import pandas as pd # импорт библиотеки pandas
from sklearn.metrics import r2_score # импорт из библиотеки sklearn алгоритма r2_score
from sklearn.linear_model import LinearRegression # # импорт из библиотеки sklearn алгоритма LinearRegression

In [2]:
# знакомство с данными
df = pd.read_csv('***.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
# приведем столбцы к стандарту PEP8
df.rename(columns = {'Пол':'gender',
                     'Возраст':'age',
                     'Зарплата':'salary',
                     'Члены семьи':'family_members',
                     'Страховые выплаты':'insurance_payments'},
                     inplace=True)

In [4]:
df.duplicated().sum() # проверка дубликатов

153

Достоверно сказать, что количество полученных дубликатов является реальными дубликатами невозможно, потому что у нас нет уникального id у клиента. А значит клиенты могут иметь идентичные параметры.  И запросить данные мы не можем, так как это учебный проект. 

На решение задачи это не повлияет, поэтому оставим данные без изменений.

In [5]:
df['insurance_payments'].value_counts() # уникальные значения целевого признака

0    4436
1     423
2     115
3      18
4       7
5       1
Name: insurance_payments, dtype: int64

Подавляющее большинство не обращалось за страховыми выплатами. А один целых 5 раз.

In [6]:
# определим признаки и целевой признак, переведем сразу в вектор
features = df.drop(columns='insurance_payments',axis=1).values
target = df['insurance_payments'].values

In [7]:
# проверка
print(features.shape)
print(target.shape)

(5000, 4)
(5000,)


Столбцы переведены по стандарту PEP8. Признаки разделены с целевым признаком и переведены в вектор. Данные готовы для дальнейшего исследования. 

## 2  Умножение матриц

**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** не изменится.

**Обоснование:**

### 2.1 Проведем расчеты собственным алгоритмом и импортированным:

In [8]:
# создание класса линейной регрессии
class LR:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = (np.linalg.inv(X.T.dot(X))).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
        
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [9]:
# метрика оригинальной таблицы
model = LR()
model.fit(features, target)
predictions = model.predict(features)
r2_original = r2_score(target, predictions)
r2_original

0.42494550286668

In [10]:
P = np.random.randint(10, size=(features.shape[1], features.shape[1])) # случайная обратимая матрица
np.linalg.inv(P) # проверка, обратима ли матрица
N = features.dot(P) # обратимую матрицу умножаем на матрицу признаков

In [11]:
# метрика преобразованной таблицы
model.fit(N, target)
predictions = model.predict(N)
r2_modified = r2_score(target, predictions)
r2_modified

0.4249455027729636

In [12]:
print('Разница метрик на собственном алгоритме:', r2_original-r2_modified)

Разница метрик на собственном алгоритме: 9.371636799926364e-11


In [13]:
# проверка на импортированном алгоритме
# метрика оригинальной таблицы
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
r2_original = r2_score(target, predictions)
r2_original

0.4249455028666801

Посмотрим на веса и сдвиг модели оригинальной таблицы:

In [14]:
model.coef_

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

In [15]:
model.intercept_

-0.9382355041528408

In [16]:
# проверка на импортированном алгоритме
# метрика преобразованной таблицы
model.fit(N, target)
predictions = model.predict(N)
r2_modified = r2_score(target, predictions)
r2_modified

0.42494550286673416

Посмотрим на веса и сдвиг модели преобразованной таблицы:

In [17]:
model.coef_

array([-0.01394464,  0.03738101, -0.03600982,  0.0277825 ])

In [18]:
model.intercept_

-0.9382355041546585

In [19]:
print('Разница метрик на импортированном алгоритме:', r2_original-r2_modified)

Разница метрик на импортированном алгоритме: -5.4067861299245124e-14


**Вывод:** 
- Предсказания почти не изменились. Совсем незначительная погрешность присутствует. Это происходит из-за умножения на обратную матрицу.
- Алгоритм из sklearn дает лучше результат, что логично.

### 2.2 Расчеты формулами 

 - $w'$ - вектор линейной регрессии, после умножения на матрицу P

С учетом того, что предсказания не поменялись с добавлением матрицы P, примем следующее равенство:

$$
a=XPw'
$$
Из этого следует:
$$
Xw=XPw'
$$
Сократим матрицу P:
$$
w=Pw'
$$
Новый вектор линейной регрессии вычисляется по формуле:
$$
w'=P^{-1}w
$$

Теперь это значение мы можем подставить в начальную формулу:
$$a=XPP^{-1}w$$
Умножение матрицы P на обратную дает единичную матрицу: 
$$a=XEw$$
А умножение любой матрицы на единичную дает ту же матрицу: 
$$a=Xw$$

Мы вернулись к изначальной формуле.

### 2.3 Расчеты формулами (v2)

Распишем предсказания оригинальной таблицы более подробно:

$$
a=Xw
$$

$$
w = (X^T X)^{-1} X^T y
$$

$$
a=X(X^T X)^{-1} X^T y
$$

Тоже самое сделаем для измененной таблицы:

$$
a'=XPw'
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$

$$
a'=XP((XP)^T XP)^{-1} (XP)^T y
$$

Здесь можно упростить формулу, с учетом того, что матрица $P$ обратима:

$$
a'=XP(P^TX^TXP)^{-1} P^TX^T y
$$

$$
a'=XPP^{-1}(X^TX)^{-1} (P^T)^{-1}P^TX^T y
$$

$$
a'=XE(X^TX)^{-1} EX^T y
$$

$$
a'=X(X^T X)^{-1} X^T y
$$

Предсказания оригинальной таблицы и измененной у нас получились тождественно равны:

$$
a=X(X^T X)^{-1} X^T y
$$

$$
a'=X(X^T X)^{-1} X^T y
$$

**Вывод:**

 - Качество линейной регрессии не изменится после умножения признаков на обратимую матрицу.
 - Нулевой элемент(w0-сдвиг) не изменится (что естественно, нулевой столбец состоит из единиц).
 - Вектор весов изменится, в зависимости от признаков, умноженных на матрицу P.


## 3 Алгоритм преобразования

**Алгоритм**

1. Создание класса шифрования признаков с двумя методами.
2. Первый метод "Шифрование":
    - Создание случайной обратимой матрицы ($P$)
    - Умножение признаков на случайную матрицу
    - Метод возвращает матрицу "шифрованных" признаков
3. Второй метод "Расшифровка":
    - Умножение "шифрованных" признаков на обратную матрицу ($P^{-1}$)
    - Округление до целых чисел
    - Приведение к абсолютным значениям.

**Обоснование**

Исходя из расчетов во втором пункте, мы выяснили, что умножение признаков на случайную обратимую матрицу, меняет признаки. Но на предсказаниях это не отражается, потому что затем умножение идет на обратную матрицу. В итоге мы получаем единичную матрицу, умножение на которую возвращает исходную.

## 4 Проверка алгоритма

In [20]:
class code:
    def coding(self, features): # шифрование признаков
        try:
            P = np.random.randint(1000, size=(features.shape[1], features.shape[1])) # случайная обратимая матрица
            np.linalg.inv(P) # проверяем, обратима ли матрица
            N = features.dot(P) # умножение на случайную матрицу
            return N
        except: # если матрица не обратима:
            return code().coding(features) #  функция будет вызываться, пока не создаст обратимую матрицу
                
    def decoding(self, features): # расшифровка
        return abs(np.around(N.dot(np.linalg.inv(P)))) # умножение на обратную случайную матрицу, округление

In [21]:
# метрика оригинальной таблицы
model.fit(features, target)
predictions = model.predict(features)
r2_original = r2_score(target, predictions)
r2_original

0.4249455028666801

In [22]:
# метрика преобразованной таблицы
coded_features = code().coding(features)
model.fit(coded_features, target)
predictions = model.predict(coded_features)
r2_modified = r2_score(target, predictions)
r2_modified

0.4249455028667193

In [23]:
print('Разница метрик до и после преобразования:', r2_original-r2_modified)

Разница метрик до и после преобразования: -3.9190872769268026e-14


In [24]:
np.allclose(code().decoding(coded_features), features) # проверка алгоритма расшифровки

True

In [25]:
coded_features # шифрованные данные

array([[13804368., 20843688., 25061963., 22137101.],
       [10581232., 15972270., 19204661., 16964880.],
       [ 5848759.,  8827656., 10613961.,  9376208.],
       ...,
       [ 9431952., 14243532., 17126574., 15127816.],
       [ 9099451., 13740924., 16522486., 14594389.],
       [11297545., 17060256., 20512946., 18118525.]])

**Вывод:**

- Качество линейной регресии не изменилось после умножения на обратимую матрицу.
- Алгоритм по расшифровке признаков работает. Возвращаются исходные значения.
- Довольно сложно будет восстановить персональную информацию, потому что признаки умножаются на случайные числа в пределах 1000.